In [ ]:
# | default_exp cli

In [ ]:
# | export

from typing import *
from pathlib import Path

import typer

import nbdev_mkdocs
import nbdev_mkdocs.mkdocs

In [ ]:
from typer.testing import CliRunner

runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")


@_app.command(
    help="Creates files in **mkdocs** subdirectory needed for other **nbdev_mkdocs** subcommands",
)
def new(root_path: str = typer.Option(".", help="")):
    """CLI command for creating files for nbdev_mkdocs command"""
    try:
        nbdev_mkdocs.mkdocs.new(root_path=root_path)
    except Exception as e:
        typer.secho("Unexpected internal error :/", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


@_app.command(
    help="Prepares files in **mkdocs/docs** and then runs **mkdocs build** command on them ",
)
def prepare(root_path: str = typer.Option(".", help="")):
    """CLI command for creating files for nbdev_mkdocs command"""
    try:
        nbdev_mkdocs.mkdocs.prepare(root_path=root_path)
    except Exception as e:
        typer.secho("Unexpected internal error :/", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

@_app.command(
    help="Prepares files in **mkdocs/docs** and then runs **mkdocs serve** command on them ",
)
def preview(root_path: str = typer.Option(".", help="")):
    """CLI command for creating files for nbdev_mkdocs command"""
    try:
        nbdev_mkdocs.mkdocs.preview(root_path=root_path, port=4000)
    except Exception as e:
        typer.secho("Unexpected internal error :/", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


In [ ]:
result = runner.invoke(_app, ["--help"])

                                                                                                                   
 Usage: root [OPTIONS] COMMAND [ARGS]...                                                                           
                                                                                                                   

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --install-completion          Install completion for the current shell.                                         │
│ --show-completion             Show completion for the current shell, to copy it or customize the installation.  │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Commands ──────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ new        Creates files in **mkdocs** subdirectory needed for other **nbdev_mkdocs** subcommands               │
│ prepare    Prepares files in **mkdocs/docs** and then runs **mkdocs build** command on them                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["new", "--help"])

                                                                                                                   
 Usage: root new [OPTIONS]                                                                                         
                                                                                                                   

Creates files in **mkdocs** subdirectory needed for other **nbdev_mkdocs** subcommands

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  [default: .]                                                                           │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["new", "--root-path=../setting.ini"])

In [ ]:
# result = runner.invoke(_app, ["prepare", "--root-path=../setting.ini"])

# assert result.exit_code == 1
# assert result.output == 'Unexpected internal error :/\n'